**Homework 3- Question 3 Terrorist Problem Solution**

The given formulation for terrorist problem in the question is as follows

\begin{equation*}
\begin{aligned}
\min_{u} \quad \{\max_{x} & \{u^TAx\} \}\\
\textrm{s.t.} \quad & u \geq 0 \\
  & x \geq 0    \\
  & e^Tx = e^Tu = 1 \\
\end{aligned}
\end{equation*}

We can rewrite the above formulation in a way that is simple to code in Pyomo (by introducing a new variable). Here is an equivalent formulation to the above problem.

\begin{equation*}
\begin{aligned}
\min_{u,w} \quad  w\\
\textrm{s.t.} \quad & we^T -u^TA \geq 0 \\
  & u^Te = 1 \\
  & u \geq 0    \\
\end{aligned}
\end{equation*}

Now, let us code this in Pyomo


Importing Libraries

In [1]:
from pyomo.environ import *
import numpy as np

Defining the model, sets and varibles

In [2]:
# Defining the Model
model = ConcreteModel()

# Sets
model.I = RangeSet(1, 3)

# Define variables
model.u = Var(model.I, domain=NonNegativeReals)
model.w = Var(domain = Reals)

Defining objective and constraints

In [3]:
# Objective
model.obj = Objective(expr= model.w, sense=minimize)

# Constraints
model.con1 = Constraint(expr=sum(model.u[j] for j in model.I) == 1)

# Define the matrix A as a dictionary
A_values = {(1, 1): 4, (1, 2): -10, (1, 3): -10, (2, 1): -8, (2, 2): 5, (2, 3): -8, (3, 1): -12, (3, 2): -12, (3, 3): 9}
model.A = Param(model.I, model.I, initialize=A_values)

def cost_constraint_rule(model, j):
    return model.w - np.sum(model.A[i,j] * model.u[i] for i in model.I) >= 0

model.con2 = Constraint(model.I, rule=cost_constraint_rule)

C:\Users\saima\AppData\Local\Temp\ipykernel_13504\64008927.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return model.w - np.sum(model.A[i,j] * model.u[i] for i in model.I) >= 0


Soving the above model using a solver( I used Gurobi)

In [4]:
solver = SolverFactory('gurobi')
solver.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': -4.598130841121495, 'Upper bound': -4.598130841121495, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 4, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.3341655731201172}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Displaying the result

In [5]:
print("Optimal value of the objective function:", model.obj())
print("Optimal values of u:")
for j in model.I:
    print(f"u[{j}] =", value(model.u[j]))

Optimal value of the objective function: -4.598130841121495
Optimal values of u:
u[1] = 0.36448598130841114
u[2] = 0.39252336448598135
u[3] = 0.24299065420560745
